In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import SelectKBest, f_classif 
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

## Importo e leggo i dati 

In [3]:
train = pd.read_csv('../Desktop/training.csv')
test = pd.read_csv('../Desktop/test.csv')

In [4]:
train

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.247937e+12,5587761,F,2016-04-15T09:03:51Z,2016-05-18T00:00:00Z,29,SANTA MARTHA,0,0,0,0,0,0,Yes
1,7.894582e+11,5774796,F,2016-06-06T08:41:45Z,2016-06-06T00:00:00Z,5,JOANA D´ARC,0,0,0,0,0,0,No
2,3.967220e+14,5605217,M,2016-04-20T08:44:47Z,2016-05-10T00:00:00Z,64,JARDIM DA PENHA,0,0,0,0,0,1,No
3,6.663786e+14,5720245,F,2016-05-19T13:31:38Z,2016-05-19T00:00:00Z,66,PRAIA DO CANTO,0,1,0,0,0,0,No
4,1.835776e+12,5742007,M,2016-05-30T07:38:58Z,2016-05-30T00:00:00Z,72,RESISTÊNCIA,0,0,0,0,0,0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66310,7.868846e+12,5721432,F,2016-05-19T16:26:59Z,2016-06-02T00:00:00Z,11,SANTOS DUMONT,0,0,0,0,0,1,No
66311,7.711876e+13,5607080,M,2016-04-20T11:03:44Z,2016-05-11T00:00:00Z,50,TABUAZEIRO,0,1,1,0,0,1,No
66312,6.293825e+14,5758142,F,2016-06-01T10:19:43Z,2016-06-01T00:00:00Z,42,ANDORINHAS,0,0,0,0,0,0,No
66313,3.824988e+12,5579699,M,2016-04-13T15:26:38Z,2016-05-05T00:00:00Z,13,BELA VISTA,0,0,0,0,0,1,Yes


In [5]:
test

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,1.167239e+11,5778236,M,2016-06-06T15:14:21Z,2016-06-08T00:00:00Z,9,INHANGUETÁ,0,0,0,0,0,0,No
1,2.375498e+13,5739813,M,2016-05-25T14:10:29Z,2016-05-30T00:00:00Z,50,ANDORINHAS,0,0,1,0,0,1,No
2,9.435712e+10,5683899,F,2016-05-11T08:00:20Z,2016-05-11T00:00:00Z,69,JARDIM CAMBURI,0,0,0,0,0,0,No
3,8.277442e+13,5688382,F,2016-05-12T07:19:39Z,2016-05-16T00:00:00Z,22,FORTE SÃO JOÃO,0,0,0,0,0,0,No
4,1.812426e+14,5707554,F,2016-05-17T09:47:04Z,2016-05-17T00:00:00Z,1,REDENÇÃO,0,0,0,0,0,0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22101,5.536134e+14,5645162,F,2016-05-02T09:33:04Z,2016-05-18T00:00:00Z,9,DO MOSCOSO,0,0,0,0,0,0,No
22102,8.873752e+14,5697393,F,2016-05-13T15:43:41Z,2016-05-20T00:00:00Z,6,CRUZAMENTO,0,0,0,0,0,0,No
22103,2.948428e+14,5622058,F,2016-04-26T11:16:09Z,2016-05-04T00:00:00Z,50,JARDIM CAMBURI,0,0,0,0,0,1,No
22104,8.622669e+13,5717386,M,2016-05-19T07:54:32Z,2016-05-19T00:00:00Z,15,JARDIM CAMBURI,0,0,0,0,0,0,No


## Divido i dati 

In [6]:
X_train_full= train.drop("No-show", axis=1).copy()
y_train = train['No-show']
X_valid_full = test.drop("No-show", axis=1).copy()
y_val = test['No-show']

## Controllo se ci sono valori nulli e i tipi 

In [7]:
X_train_full.isnull().sum()

PatientId         0
AppointmentID     0
Gender            0
ScheduledDay      0
AppointmentDay    0
Age               0
Neighbourhood     0
Scholarship       0
Hipertension      0
Diabetes          0
Alcoholism        0
Handcap           0
SMS_received      0
dtype: int64

In [8]:
X_train_full.dtypes

PatientId         float64
AppointmentID       int64
Gender             object
ScheduledDay       object
AppointmentDay     object
Age                 int64
Neighbourhood      object
Scholarship         int64
Hipertension        int64
Diabetes            int64
Alcoholism          int64
Handcap             int64
SMS_received        int64
dtype: object

## Definisco quali feature eliminare 

In [9]:
features_da_eliminare = ["PatientId", "AppointmentID"]
new_X_train_full=X_train_full.drop(features_da_eliminare, axis=1, inplace=True)
new_X_valid_full=X_valid_full.drop(features_da_eliminare, axis=1, inplace=True)

## Controllo quanti valori ho per le date

In [10]:
print('AppointmentDay:',sorted(X_train_full.AppointmentDay.unique()))

AppointmentDay: ['2016-04-29T00:00:00Z', '2016-05-02T00:00:00Z', '2016-05-03T00:00:00Z', '2016-05-04T00:00:00Z', '2016-05-05T00:00:00Z', '2016-05-06T00:00:00Z', '2016-05-09T00:00:00Z', '2016-05-10T00:00:00Z', '2016-05-11T00:00:00Z', '2016-05-12T00:00:00Z', '2016-05-13T00:00:00Z', '2016-05-14T00:00:00Z', '2016-05-16T00:00:00Z', '2016-05-17T00:00:00Z', '2016-05-18T00:00:00Z', '2016-05-19T00:00:00Z', '2016-05-20T00:00:00Z', '2016-05-24T00:00:00Z', '2016-05-25T00:00:00Z', '2016-05-30T00:00:00Z', '2016-05-31T00:00:00Z', '2016-06-01T00:00:00Z', '2016-06-02T00:00:00Z', '2016-06-03T00:00:00Z', '2016-06-06T00:00:00Z', '2016-06-07T00:00:00Z', '2016-06-08T00:00:00Z']


In [11]:
## Ho notato il formato strano delle date e ho deciso di trasformarle nel formato data corretto
X_train_full.ScheduledDay = X_train_full.ScheduledDay.apply(np.datetime64)
X_train_full.AppointmentDay = X_train_full.AppointmentDay.apply(np.datetime64)

## Controllo che gli appuntamenti non siano in giorni feriali
from datetime import date
import calendar
my_date = date(2016, 6, 2)
calendar.day_name[my_date.weekday()]
                                     
## avevo pensato di calcolare il numero di giorni che passavano da quando veniva preso l'aapuntamento a quando effettivamente 
## c'era l'appuntamento e di memorizzare quel dato nella colonna daysToAppointment ma ho notato che l'accuracy si abbassava

#daysToAppointment = X_train_full.AppointmentDay - X_train_full.ScheduledDay
#daysToAppointment = daysToAppointment.apply(lambda x: x.total_seconds() / (3600 * 24))

#X_train_full.insert(2, 'daysToAppointment', daysToAppointment)
#X_valid_full.insert(2, 'daysToAppointment', daysToAppointment)
#X_train_full.daysToAppointment = X_train_full.daysToAppointment.astype('int64')

'Thursday'

In [12]:
X_train_full

,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received
0,F,2016-04-15 09:03:51,2016-05-18,29,SANTA MARTHA,0,0,0,0,0,0
1,F,2016-06-06 08:41:45,2016-06-06,5,JOANA D´ARC,0,0,0,0,0,0
2,M,2016-04-20 08:44:47,2016-05-10,64,JARDIM DA PENHA,0,0,0,0,0,1
3,F,2016-05-19 13:31:38,2016-05-19,66,PRAIA DO CANTO,0,1,0,0,0,0
4,M,2016-05-30 07:38:58,2016-05-30,72,RESISTÊNCIA,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
66310,F,2016-05-19 16:26:59,2016-06-02,11,SANTOS DUMONT,0,0,0,0,0,1
66311,M,2016-04-20 11:03:44,2016-05-11,50,TABUAZEIRO,0,1,1,0,0,1
66312,F,2016-06-01 10:19:43,2016-06-01,42,ANDORINHAS,0,0,0,0,0,0
66313,M,2016-04-13 15:26:38,2016-05-05,13,BELA VISTA,0,0,0,0,0,1


In [13]:
# Per poterle passare alla pipeline e poterci allenare il modello ho deciso di cambiare il tipo delle date di nuovo
X_train_full.ScheduledDay = X_train_full.ScheduledDay.astype('object')
X_train_full.AppointmentDay = X_train_full.AppointmentDay.astype('object')

In [14]:
# Selezionato le colonne categoriche 
categorical_cols = [cname for cname in X_train_full.columns if 
                        X_train_full[cname].dtype == "object" ]

# Selezionato le colonne numeriche 
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]


my_cols = categorical_cols + numerical_cols 
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [15]:
#Controllo se le colonne con variabili categoriche sono state memorizzate in categorical_cols
categorical_cols

['Gender', 'ScheduledDay', 'AppointmentDay', 'Neighbourhood']

In [16]:
#Controllo se le colonne con valori numerici sono state memorizzate in numerical_cols
numerical_cols

['Age',
 'Scholarship',
 'Hipertension',
 'Diabetes',
 'Alcoholism',
 'Handcap',
 'SMS_received']

## Preparo i dati che passerò alla pipeline

In [17]:

# Preprocessamento dei dati numerici
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])



# Preprocessamento dei dati categorici
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
       
    ])

## Dichiaro il modello che andrò ad utilizzare nella pipeline

In [18]:
appuntamento_model=LogisticRegression(max_iter=1000, random_state=0)

## Pipeline con allenamento e predizione 

In [19]:
# Lego il preprocessamento dei dati al modello e sfrutto skb per selezionare le feature migliori
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('skb', SelectKBest(f_classif, k = 11)),
                              ('appuntamento_model', appuntamento_model)
                             ])

# Preprocessamento dei dati di training e allenamento del modello
my_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                             

In [20]:
# Predizione
preds = my_pipeline.predict(X_valid)

## Accuracy

In [21]:
acc_appuntamento_model=accuracy_score(y_val,preds)
acc_appuntamento_model

0.8019542205735999